In [40]:
import pandas as pd

In [41]:
path = 'https://raw.githubusercontent.com/EmmS21/SpringboardCapstoneBoxingPredictionWebApp/master/boxingdata/visuals.csv'
path_two = 'https://raw.githubusercontent.com/EmmS21/SpringboardCapstoneBoxingPredictionWebApp/master/boxingdata/longformateddata2.csv'
data = pd.read_csv(path)
data['division'] = data['division'].fillna('unknown')
data['bouts_fought'] = data['w'].astype('float')+data['l'].astype('float')+data['d'].astype('float')
#uploading data for heatmap
long_data = pd.read_csv(path_two)
#count wins by stance
df = long_data[long_data['outcome'].str.contains('win',na=False)].groupby(['stance1', 'stance.y','division','sex'])['outcome'].count().reset_index()
#dataframe of total wins,losses and draws by age range
wins = long_data[long_data['outcome'].str.contains('win',na=False)].groupby(['age_range', 'opp_age_range','division','sex'])['outcome'].count().reset_index()
loss = long_data[long_data['outcome'].str.contains('loss',na=False)].groupby(['age_range', 'opp_age_range','division','sex'])['outcome'].count().reset_index()
draw = long_data[long_data['outcome'].str.contains('draw',na=False)].groupby(['age_range', 'opp_age_range','division','sex'])['outcome'].count().reset_index()
#other fight outcomes - nan, unknown etc
outcomes = ['win','draw','loss']
outcomeslist = '|'.join(outcomes)
other = long_data[~long_data['outcome'].str.contains(outcomeslist,na=False)].groupby(['age_range', 'opp_age_range','division','sex'])['outcome'].count().reset_index()
#merge all outcomes
fight_outcomes = pd.DataFrame()
fight_outcomes[['age_range','opp_age_range','division','sex','wins']] = wins[['age_range','opp_age_range','division','sex','outcome']]
fight_outcomes = fight_outcomes.merge(loss, on=['age_range','opp_age_range','division','sex']).rename(columns={'outcome':'loss'}).merge(draw, on=['age_range','opp_age_range','division','sex']).rename(columns={'outcome':'draw'}).merge(other, on=['age_range','opp_age_range','division','sex']).rename(columns={'outcome':'other'})
#calculate win rates by different age groups
fight_outcomes['total_fights'] = fight_outcomes[['wins','loss','draw','other']].sum(axis=1)
fight_outcomes['win_rate']=(fight_outcomes['wins']/fight_outcomes['total_fights'])*100
#count total fights by stance
df2 = long_data[['division','outcome']].groupby('division').count().reset_index()
df2.rename(columns={'outcome':'total'},inplace=True)
df = df.merge(df2,on='division')
#calculate wins in percentages
df['win_rate'] =df['outcome']/df['total'] *100
#filter out where stance1 is unknown
df = df[df.stance1 != 'unknown']
#cleaning data for top boxer table
firstBoxer = ['firstBoxerWeight'+str(i) for i in range(1,85)]
secondBoxer = ['secondBoxerWeight'+str(i) for i in range(1,85)]
opp_rating = ['secondBoxerRating'+str(i) for i in range(1,85)]
#average weight of boxer
data['average_weight'] = data[firstBoxer].mean(axis=1)
#average opponent weight
data['average_opponent_weight'] = data[secondBoxer].mean(axis=1)
#extract wins and losses
def wins(col):
    return data[col].astype(str).str.extract('win(?P<win>.*?)}')
def draws(col):
    return data[col].astype(str).str.extract('draw(?P<draw>.*?)l')
opp_rank = ['secondBoxerRecord'+str(i) for i in range(1,85)]
#update columns with wins and losses
for i in opp_rank:
    data['opp_wins'+str(i)] = wins(i)
for i in opp_rank:
    data['opp_draws'+str(i)] = draws(i)
opp_drawsd = ['opp_draws'+str(i) for i in opp_rank]
opp_winsd = ['opp_wins'+str(i) for i in opp_rank]
#remove quotation marks
data[opp_drawsd] = data[opp_drawsd].apply(lambda x: x.str.replace('"',''))
data[opp_winsd] = data[opp_drawsd].apply(lambda x: x.str.replace('"',''))
#replace all letters with number 0
data[opp_drawsd] = data[opp_drawsd].replace(regex='([a-zA-Z])', value=0)
data[opp_winsd] = data[opp_winsd].replace(regex='([a-zA-Z])', value=0)
#assume win is 10 points and draw is 5 points
data[opp_winsd] = data[opp_winsd].astype(float)*10
data[opp_drawsd] = data[opp_drawsd].astype(float)*5
#new column with total points
data['opp_points'] = data[opp_winsd].sum(axis=1) + data[opp_drawsd].sum(axis=1)
#divide points by number of bouts fought
data['opp_points'] = data['opp_points']/data['bouts_fought']
#get type of wins from referee column
ref_outcome = ['referee'+str(i) for i in range(1,85)]
topten = pd.concat([data,data[ref_outcome].astype(str).stack().str.replace('\[|\"','').str.extract('(\w+\s\w+)').groupby(level=0)[0].apply(pd.Series.value_counts).unstack(fill_value=0)],axis=1)
topten = topten[['name','w','d','l','location','division','average_weight','average_opponent_weight','opp_points','bouts_fought','win KO','win SD', 'win TKO', 'win UD','sex']]
topten.rename(columns={'w':'wins','d':'draws','l':'losses','win KO':'win by knockout','win SD':'win by split decision','win TKO':'win by technical knockout','win UD':'win by unanimous decision'},inplace=True)
topten['total_points'] = ((topten['wins']*10 +(topten['draws']*5) - (topten['losses']*25)))+topten['opp_points']
opp_names = ['secondBoxer'+str(i) for i in range(1,85)]
outcome_cols = ['outcome'+str(i) for i in range(1,85)]
#remove quotation marks from secondboxer name
data[opp_names] = data[opp_names].astype(str).apply(lambda x: x.str.replace('"',''))
#get points for each opponent, if negative convert to zero
data[opp_names]=data[opp_names].apply(lambda x : x.map(dict(zip(topten.name,topten.total_points))))
data[opp_names] = data[opp_names].fillna(0)
data[opp_names] = data[opp_names].mask(data[opp_names] < 0, 0)
data[outcome_cols] = data[outcome_cols].astype(str).apply(lambda x: x.str.replace('"',''))
#add opp points to total points if outcome was a win
topten['total_points'] = (data[opp_names].where(data[outcome_cols].eq('win ').values, 0).sum(axis=1)/5) + (topten['total_points'])
topten.to_csv("C://Users//User//Documents//GitHub//SpringboardCapstoneBoxingPredictionWebApp//boxingdata//topten.csv")
data.to_csv("C://Users//User//Documents//GitHub//SpringboardCapstoneBoxingPredictionWebApp//boxingdata//vizdata.csv")
df.to_csv("C://Users//User//Documents//GitHub//SpringboardCapstoneBoxingPredictionWebApp//boxingdata//df.csv")
fight_outcomes.to_csv("C://Users//User//Documents//GitHub//SpringboardCapstoneBoxingPredictionWebApp//boxingdata//fightoutcomes.csv")

C:\Users\User\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (697,700,717,734,751,759,760,762,763,764,765,766,767,768,769,770,771,772,773,775,776,777,779,780,781,782,783,784,785,786,787,788,789,790,792,793,794,796,797,798,799,800,801,802,803,804,805,806,807,809,810,811,813,814,815,816,817,818,819,820,821,822,823,824,826,827,828,830,831,832,833,834,835,836,837,838,839,840,841,843,844,845,847,848,849,850,851,852,853,854,855,856,857,858,860,861,862,864,865,866,867,868,869,870,871,872,873,874,875,877,878,879,881,882,883,884,885,886,887,888,889,890,891,892,894,895,896,898,899,900,901,902,903,904,905,906,907,908,909,911,912,913,915,916,917,918,919,920,921,922,923,924,925,926,928,929,930,932,933,934,935,936,937,938,939,940,941,942,943,945,946,947,949,950,951,952,953,954,955,956,957,958,959,960,962,963,964,966,967,968,969,970,971,972,973,974,975,976,977,979,980,981,983,984,985,986,987,988,989,990,991,992,993,994,996,997,998,1000,1001,1002,1003,1004,100